In [1]:
import os
import sys
import openai
sys.path.append('/home/ec2-user/Bigdata-Spring2023-TeamHoWan/')
from spatial_profiler.resource.datamart_API import DataMartAPI
from spatial_profiler.resource.geoconcat import GeoConcat
from spatial_profiler.resource.profiler import SpatialProfiler

# openai.organization = "org-"
# OPENAI_API_KEY="sk-"
OPENAI_API_KEY=os.getenv("THW_GPT_KEY")
openai.api_key = OPENAI_API_KEY
model_list = openai.Model.list()

/home/ec2-user/.local/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
dm = DataMartAPI()
dm.search("New York, spatial")
dm.next_page()
single_data = dm.get_single_result(3)

In [3]:
# import locationtagger
dm = DataMartAPI()
dm.search("New York, Spatial")
dm.next_page()
single_data = dm.get_single_result(1)
addr=dm.get_descriptions(single_data)
addr, dm.get_column_names(single_data)

('Synthesis of Environmental Mercury Loads in New York State (1969-2017): Sites. Description: This dataset includes information from a statewide synthesis effort, involving the compilation, analysis, and synthesis of existing data on environmental mercury (Hg) concentrations and loads in New York State, with an emphasis on Hg data collected during recent NYSERDA-sponsored monitoring and research efforts. Compiling existing mercury data is a vital step to understanding the broad spatial and temporal patterns of mercury across the landscape. This dataset includes information about the locations in which samples were collected and is one of eight datasets from the statewide synthesis database.  \n\nThe New York State Energy Research and Development Authority (NYSERDA) offers objective information and analysis, innovative programs, technical expertise, and support to help New Yorkers increase energy efficiency, save money, use renewable energy, and accelerate economic growth. reduce relian

In [4]:
geo_key = os.getenv("THW_GEO_KEY")
profiler = SpatialProfiler(geo_key=geo_key)
# df = profiler.profiling(single_data)
profiler.geoconcat.geocoding_all("11201")

[{'address_components': [{'long_name': '11201',
    'short_name': '11201',
    'types': ['postal_code']},
   {'long_name': 'Brooklyn Heights',
    'short_name': 'Brooklyn Heights',
    'types': ['neighborhood', 'political']},
   {'long_name': 'New York',
    'short_name': 'New York',
    'types': ['locality', 'political']},
   {'long_name': 'New York',
    'short_name': 'NY',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']}],
  'formatted_address': 'Brooklyn Heights, NY 11201, USA',
  'geometry': {'bounds': {'northeast': {'lat': 40.710594, 'lng': -73.9688331},
    'southwest': {'lat': 40.6835594, 'lng': -74.006444}},
   'location': {'lat': 40.6986772, 'lng': -73.9859414},
   'location_type': 'APPROXIMATE',
   'viewport': {'northeast': {'lat': 40.710594, 'lng': -73.9688331},
    'southwest': {'lat': 40.6835594, 'lng': -74.006444}}},
  'place_id': 'ChIJ10KLdUlawokRxmIQu1GuCHk',
  '

In [53]:
import numpy as np
import pandas as pd
def assemble_single_row(row):
    # 1. check lat long
    if row.filter(regex="_longitude$").shape[0] and  row.filter(regex="_latitude$").shape[0]:
        lat = row.filter(regex="_latitude$").values[0]
        long = row.filter(regex="_longitude$").values[0]
        if not pd.isna(lat) and not pd.isna(long):
            return f"({lat}, {long})"
        
    # 2. check geo
    if row.filter(regex="_geojson$").shape[0]:
        geo = row.filter(regex="_geojson$").values[0]
        if not pd.isna(geo):
            return f"({geo[1]}, {geo[0]})"
        
    # 3. check postal
    if row.filter(regex="_zipcode$").shape[0]:
        zipcode = row.filter(regex="_zipcode$").values[0]
        if not pd.isna(zipcode):
            return zipcode
    
    # address
    return row["TeamHoWan_address"]
    
    
df.apply(assemble_single_row, axis=1)

0                          Fishkill, NY
1                    Big Moose Lake, NY
2                    Castleton Isl., NY
3                 St. Lawrence Riv., NY
4      RO1880, NY, Rockland, Mid-Hudson
                     ...               
495               (42.94306, -75.92278)
496               (42.28297, -78.67303)
497                (41.0456, -72.29373)
498                (42.9033, -73.34961)
499                 (42.93944, -73.285)
Length: 500, dtype: object

In [4]:
df

,Capture_Loc,Project_ID,Project_Title,SiteName,TerritoryName,Latitude,Longitude,Waterbody_Type,State,County,Region,Georefence,TeamHoWan_address,TeamHoWan_Latitude_latitude,TeamHoWan_Longitude_longitude,TeamHoWan_Georefence_geojson,thw_parsed,thw_parsed_geo
0,NY013-NY044NYT190-,NY013,BRI NY Bat Data,Fishkill,Fishkill,,,,NY,,,,"Fishkill, NY",NaN,NaN,None,"Fishkill, NY","(-73.89902520000001, 41.53564919999999)"
1,NY014--,NY014,BRI NY Loon Data,Big Moose Lake,West Bay,,,,NY,,,,"Big Moose Lake, NY",NaN,NaN,None,"Big Moose Lake, NY","(-74.849251, 43.8315505)"
2,NY015-Castleton Isl.-,NY015,BRI NY BAEA Data,Castleton Isl.,,,,,NY,,,,"Castleton Isl., NY",NaN,NaN,None,"Castleton Isl., NY","(-73.2103934, 40.7298207)"
3,NY015-St. Lawrence Riv.-,NY015,BRI NY BAEA Data,St. Lawrence Riv.,,,,,NY,,,,"St. Lawrence Riv., NY",NaN,NaN,None,"St. Lawrence Riv., NY","(-74.0059728, 40.7127753)"
4,NY016-410621073580101-00,NY016,waterdata.usgs.gov,RO1880,,,,Water-supply treatment plant,NY,Rockland,Mid-Hudson,,"RO1880, NY, Rockland, Mid-Hudson",NaN,NaN,None,"RO1880, NY, Rockland, Mid-Hudson","(-74.0312737, 41.1416845)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,NY016-425635075552201-42.94306-75.92278,NY016,waterdata.usgs.gov,OD1900,,42.94306,-75.92278,Well,NY,Onondaga,Central Ny,POINT (-75.92278 42.94306),"OD1900, NY, Onondaga, Central Ny",42.94306,-75.92278,"(-75.92278, 42.94306)","42.94306, -75.92278","(-75.92278000000002, 42.9431608)"
496,NY016-421659078402301-42.28297-78.67303,NY016,waterdata.usgs.gov,CT 472,,42.28297,-78.67303,Well,NY,Cattaraugus,Western Ny,POINT (-78.67303 42.28297),"CT 472, NY, Cattaraugus, Western Ny",42.28297,-78.67303,"(-78.67303, 42.28297)","42.28297, -78.67303","(-78.6738586, 42.2824249)"
497,NY001-NY105NYT333-41.04560-72.29373,NY001,BRI NY Birds,Mashomack Preserve,Pine Swamp Manor House,41.0456,-72.29373,,NY,Suffolk,Long Island,POINT (-72.29373 41.0456),"Mashomack Preserve, NY, Suffolk, Long Island",41.04560,-72.29373,"(-72.29373, 41.0456)","41.0456, -72.29373","(-72.29401, 41.0453404)"
498,NY002-NY-HOOSIC RIVER-42.903302-73.349606,NY002,New York State Department of Environmental Con...,HOOSIC RIVER,,42.9033,-73.34961,,NY,Rensselaer,Capital,POINT (-73.34961 42.9033),"HOOSIC RIVER, NY, Rensselaer, Capital",42.90330,-73.34961,"(-73.34961, 42.9033)","42.9033, -73.34961","(-73.348902, 42.903505)"


In [7]:
print(df.iloc[495]["thw_parsed"])
#42.94306, -75.92278
profiler.geoconcat.geocoding_all("11201")

(42.94306, -75.92278)


[{'address_components': [{'long_name': '11201',
    'short_name': '11201',
    'types': ['postal_code']},
   {'long_name': 'Brooklyn Heights',
    'short_name': 'Brooklyn Heights',
    'types': ['neighborhood', 'political']},
   {'long_name': 'New York',
    'short_name': 'New York',
    'types': ['locality', 'political']},
   {'long_name': 'New York',
    'short_name': 'NY',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']}],
  'formatted_address': 'Brooklyn Heights, NY 11201, USA',
  'geometry': {'bounds': {'northeast': {'lat': 40.710594, 'lng': -73.9688331},
    'southwest': {'lat': 40.6835594, 'lng': -74.006444}},
   'location': {'lat': 40.6986772, 'lng': -73.9859414},
   'location_type': 'APPROXIMATE',
   'viewport': {'northeast': {'lat': 40.710594, 'lng': -73.9688331},
    'southwest': {'lat': 40.6835594, 'lng': -74.006444}}},
  'place_id': 'ChIJ10KLdUlawokRxmIQu1GuCHk',
  '

In [6]:
single_data.__class__

bound = {"latitude": (out.bbox['southwest'][0], out.bbox['northeast'][0]), "longitude": (out.bbox['southwest'][1], out.bbox['northeast'][1])}

NameError: name 'out' is not defined

In [2]:
mapAPI = os.getenv("THW_GEO_KEY")
geoconcat = GeoConcat(mapAPI)
geoconcat.read_address("Planning Areas. Description: San Francisco Planning Department's Planning Areas.")

[]


In [4]:
import geograpy
from geograpy import extraction


e = extraction.Extractor(text=addr)
e.find_geoEntities()
print(e.places)

['Synthesis', 'New York State', 'New York State', 'New York', 'Facebook']


In [5]:
import geocoder
mapAPI = os.getenv("THW_GEO_KEY")

lookup_res = [geocoder.google(p, key=mapAPI) for p in e.places]
lookup_res = [i for i in lookup_res if i is not None]
lookup_res = [res for res in lookup_res if res.state is not None or res.city is not None]

lookup_res

[]

In [11]:
from collections import Counter
lookup_res_count_states = [x.state for x in lookup_res]
# Counter(lookup_res_for_count).keys() # equals to list(set(words))
# Counter(lookup_res_for_count).values() # counts the elements' frequency

state = Counter(lookup_res_count_states).most_common(1)[0][0]
lookup_res = [res for res in lookup_res if res.state == state]

lookup_res_count_cities = [x.city for x in lookup_res]
city = Counter(lookup_res_count_cities).most_common(1)[0][0]

if city:
    out = geocoder.google(f"{state}, {city}", key=mapAPI)
else:
    out = geocoder.google(f"{state} State", key=mapAPI)

out.bbox['southwest'][1], out.bbox['northeast'][1]

(-79.7625901, -71.777491)

In [44]:
res.latlng

[('AR', 1)]